<img src="./images/DLI_Header.png" style="width: 400px;">


# 4. 분산 학습 최적화

이번 노트북에서는 [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) 를 활용하여 GPT 사전 훈련 성능을 정량화하는 방법에 대해 배우고 혼합 정밀도, 그래디언트 누적 및 활성화 체크포인팅과 같은 최적화 기술을 살펴보겠습니다.

## 목표

이번 노트북의 목표는 다음과 같습니다:
* 자동 혼합 정밀도(AMP, Automatic Mixed Precision) 및 활성화 체크포인팅(Activation Checkpointing)을 사용하여 Megatron-LM 스크립트의 멀티 노드 학습을 최적화합니다. 
* 학습 성과를 계산하는 방법을 이해합니다.


**[4.1 혼합 정밀도 학습](#1.1-The-hardware-overview)<br>**
**[4.2 활성화 체크포인팅 ](#1.1-The-hardware-overview)<br>**
**[4.3 그래디언트 누적](#1.1-The-hardware-overview)<br>**
**[4.4 학습 성능](#4.4-Compute-The-Training-Performance)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[4.4.1 매개 변수의 숫자를 연산하기](#1.1.1-Check-The-Available-CPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.4.2 연습: 우리 모델의 매개 변수 숫자를 연산하기](#1.1.2-Check-The-Available-GPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.4.3 GPU 당 이론 피크 FLOP/초 연산하기](#1.1.3-Check-The-Interconnect-Topology)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.4.4 학습 기간 / 에포크를 추정하기](#1.1.3-Check-The-Interconnect-Topology)<br>

### 이전 실행/보류 중인 작업을 취소합니다.

다음으로 이동하기 전에 SLURM 대기열에서 아직 실행 중이거나 대기 중인 작업이 없는지 확인하십시오. 다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다:

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하고 `scancel` 명령을 사용하여 모든 사용자의 작업을 취소합니다.

In [2]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 4.1 혼합 정밀도 트레이닝 

<img src="images/AMP.png" width="700"/>

**Automatic Mixed Precision (AMP)** 는 수학적 연산을 실행할 때 서로 다른 수치 정밀도를 사용할 수 있습니다. 네트워크의 중요한 부분에서 단일 정밀도를 유지하면서 필요한 메모리를 줄이는 반-정밀도 형식으로 몇몇 작업을 수행합니다.

Automatic Mixed Precision (자동 혼합 정밀도)을 사용한 학습은 Volta 아키텍처(Volta, Turing, Ampere 및 그 이후 아키텍처)부터 사용할 수 있는 NVIDIA GPU 의 Tensor Core가 제공하는 하드웨어 가속화를 활용합니다. AMP 학습에 대한 자세한 내용은 [혼합 정밀도 훈련 설명서](https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html)를 참조하십시오.

이전 노트북에서는 베이스라인 실행에 대해 혼합 정밀도 트레이닝이 활성화되지 않았습니다. GPU Kernel view에서 세부 정보를 볼 수 있습니다. 

<img src="images/profiling3.png" width="650"/>

overview 탭 ( 3.4.2 Tensorboard with Pytorch Profiler 섹션의 그림에도 표시)에서 실행한 베이스라인 실행의 성능 권장 사항에 따르면 전체 시간의 27%를 차지하는 커널은 Tensor Core 적격 오퍼레이터에 의해 실행됩니다. FP16으로 자동 혼합 정밀도(AMP)를 활성화하면 연산 속도를 높일 수 있습니다.

혼합 정밀도에서 Megatron-LM 사전 훈련을 실행하려면 GPT_ARGS에 `--fp16` 인수를 추가하십시오.

In [3]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_increase_MBS_fp16.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4      
GLOBAL_BATCH_SIZE=16    

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_increase_GBS_fp16"        

OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            --fp16 \
            $OPTIMIZER_ARGS \
            "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name fp16 \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_increase_MBS_fp16.sh


이제 이전에 작업해 둔 sbatch 스크립트[pretrain_gpt_2Node4GPU_increase_MBS_fp16.sh](./code/pretrain_gpt_2Node4GPU_increase_MBS_fp16.sh) 를 제출하겠습니다. `squeue` 명령어를 사용하여 SLURM 대기열을 확인합니다.

In [4]:
# submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_increase_MBS_fp16.sh

# Check the SLURM queue
!squeue

Submitted batch job 13
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                13  slurmpar dli_2nod    admin  R       0:00      2 slurmnode[1-2]


Megatron GPT3 사전 훈련의 성능을 이해하기 위해, 우리는 실행 중에 생성된 로그를 확인할 수 있습니다.

먼저 생성된 [로그](./megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16.txt) 를 사용하여 실행의 world size를 확인합니다. 아래 부분을 확인해 보아야 합니다:

```
    using world size: 4, data-parallel-size: 4, tensor-model-parallel size: 1, pipeline-model-parallel size: 1 
    using torch.float16 for parameters ...
```

fp16 모드에서 실행할 때 `torch.float16 for parameters`라는 메시지가 나타납니다.

잠시 후 학습 성과를 확인하고 강사와 함께 논의해 봅니다. 처음으로 Megatron-LM을 실행하는 경우, 코드를 컴파일하는 데 약 6분이 소요됩니다. 그 때까지는 GPU 활동을 볼 수 없을 것입니다.

In [5]:
! grep elapsed /dli/megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16.txt

 iteration       10/     100 | consumed samples:          160 | elapsed time per iteration (ms): 509.6 | learning rate: 0.000E+00 | global batch size:    16 | loss scale: 8388608.0 | number of skipped iterations:  10 | number of nan iterations:   0 |
 iteration       20/     100 | consumed samples:          320 | elapsed time per iteration (ms): 311.9 | learning rate: 3.000E-05 | global batch size:    16 | lm loss: 1.070710E+01 | loss scale: 131072.0 | grad norm: 4.466 | number of skipped iterations:   6 | number of nan iterations:   0 |
 iteration       30/     100 | consumed samples:          480 | elapsed time per iteration (ms): 315.8 | learning rate: 5.999E-05 | global batch size:    16 | lm loss: 1.021160E+01 | loss scale: 65536.0 | grad norm: 2.516 | number of skipped iterations:   1 | number of nan iterations:   0 |
 iteration       40/     100 | consumed samples:          640 | elapsed time per iteration (ms): 318.3 | learning rate: 5.995E-05 | global batch size:    16 | lm lo

### Tensorboard - Pytorch Profiler

프로파일링은 이전 실행의 Tensorboard 링크 내 `pytorch_profiler` 탭에서 가능합니다. 텐서보드 페이지를 이미 닫은 경우 다음 셀을 실행하여 링크를 다시 생성할 수 있습니다. 링크를 클릭하여 텐서보드를 연 다음 `PYTORCH_PROFILER` 탭으로 이동합니다.

In [6]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

`run_fp16_gpu0`결과를 보면 GPU 활용도가 감소합니다. GPU 커널 뷰에서 혼합 정밀도를 활성화하면 GPU 연산의 23.6%가 텐서코어로 가속된다는 것을 알 수 있습니다.

<img src="images/profiling5_AMP.png" width="400"/>


### 메모리는 어떤가요?
12G (베이스라인에서 ~10G)의 피크에 따라 메모리 소비가 증가한다는 것을 알 수 있습니다. 이는 일부 모델 가중치가 FP32와 FP16에 모두 저장되고 액티베이션과 그레디언트는 FP16에 저장되기 때문입니다. 따라서 가중치의 메모리 사용량이 증가하는 반면 정방향 및 역방향 패스의 액티베이션 및 그레디언트에 사용되는 메모리는 줄어듭니다. 새 점프를 확대하면 추가 Pytorch 복사 연산 *_to_copy* 가 표시됩니다.

<img src="images/profiling_FP16_memory.png" width="800"/>

좋습니다! 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [7]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/* 

---
# 4.2 활성화 체크포인팅


<img src="images/activation_checkpoiting.png" width="700" align="center"/>

활성화 체크 포인팅은 추가 재계산 비용으로 학습 중에 메모리를 절약할 수 있는 또 다른 기술입니다. 바닐라 포워드, 백워드 패스에서 모든 피쳐 맵은 포워드 패스 동안 계산되고 백워드 단계를 위해 저장됩니다. 활성화 체크포인팅 전략에서는 포워드 패스 중에 일부 중간 결과만 저장합니다 (체크포인트라고 불림). 이러한 체크포인트는 필요할 때 추가 피쳐 맵을 재계산하기 위해 백워드 패스에서 사용됩니다. 체크포인팅에는 사용자가 제공하는 수동 체크포인트 또는 자동 선택을 포함하여 여러 가지 구현 방법이 있습니다.

Megatron-LM은 Uniform 및 Block의 두 가지 활성화 체크포인트 방법을 지원합니다.

- Uniform: 레이어를 균일하게 그룹으로 나누고 각 그룹의 입력 활성화를 메모리에 저장합니다. 
- Block: 파이프라인 단계당 설정된 개별 Transformer 레이어 세트의 입력 활성화를 검사합니다.

활성화 체크포인팅을 사용하여 Megatron-LM 사전 훈련을 실행하려면 단순히 --activations-checkpoint-method` 인자를 추가하여 GPT_ARGS에서 uniform 또는 block 로 설정하면 됩니다.

In [11]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4      
GLOBAL_BATCH_SIZE=16    

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing"        


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            --fp16 \
            --activations-checkpoint-method uniform \
            $OPTIMIZER_ARGS \
            "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name  fp16_activation_checkpointing \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing.sh


이제 이전에 제작한 sbatch 스크립트[pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing.sh](/dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpoiting.sh) 를 제출합니다. `squeue`명령어를 사용하여 SLURM 대기열을 확인합니다.

In [9]:
# Submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing.sh

# Check the SLURM queue
!squeue

Submitted batch job 14
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                14  slurmpar dli_2nod    admin  R       0:00      2 slurmnode[1-2]


메가트론 GPT3 사전 훈련의 성능을 이해하기 위해 생성된 [로그](./megatron/logs/log_2Nodes4GPUS_increaseBS_fp16_activation_checkpointing.txt) 를 체크하고 강사와 논의해 봅시다.

In [10]:
! grep elapsed /dli/megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing.txt

### 메모리는 어떤가요?

프로파일링은 텐서보드 링크의 `pytorch_profiler`탭에서 사용할 수 있습니다. 활성화 체크포인팅를 사용할 때 `run_fp16_activation_checkpointing_GPU0` 를 확인해 보면 메모리 사용량이 ~3G 피크까지 상당히 감소하는 것을 알 수 있습니다. 이는 일부 활성화가 저장되지 않고 필요할 때 다시 계산되기 때문입니다.
그래프를 확대하면 Pytorch CheckpointFunctions를 추적할 수 있습니다.

<img src="images/profiling_FP16_checkpoiting_memory.png" width="900"/>

좋습니다! 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [12]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/*

---
# 4.3 그래디언트 축적 


배치 크기를 늘리는 또 다른 방법은 그래디언트 축적(Gradient Accumulation)을 사용하는 것입니다. 분산 데이터 병렬에서처럼 워커 간에 데이터를 분할하는 대신, 모델의 매개 변수를 업데이트하기 전에 동일한 워커가 여러 배치를 처리하고 그래디언트을 누적합니다.

[NVIDIA APEX Library](https://github.com/NVIDIA/apex#quick-start) 는 자동 혼합 정밀도로 그래디언트 축적을 사용할 때 최적화된 구현 방식을 제공합니다. 이 구현 방식은 그래디언트의 불필요한 이중 정밀도 복사본들을 제거하기 위해 먼저 낮은 정밀도로 축적한 후 다시 이중 정밀도로 돌아갑니다. Megatron-LM  라이브러리는 APEX 구현을 사용합니다.  


Gradient Accumulation(그래디언트 누적)을 사용하여 Megatron-LM 사전 훈련을 실행하려면 마이크로 배치 크기를 동일하게 유지하면서 글로벌 배치 크기를 늘리기만 하면 됩니다.  

In [13]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing_gradient_accumulation.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4      
GLOBAL_BATCH_SIZE=64    

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing_gradient_accumulation"        


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            --fp16 \
            --activations-checkpoint-method uniform \
            $OPTIMIZER_ARGS \
            "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name fp16_activation_checkpointing_gradient_accumulation \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Writing /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing_gradient_accumulation.sh


이제 이전의 sbatch 스크립트 [pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing_gradient_accumulation.sh](/dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing_gradient_accumulation.sh)를 실행합니다. `squeue` 명령어를 사용하여 SLURM 대기열을 확인합니다.

In [14]:
# Submit the 2 nodes jobs
!sbatch /dli/code/pretrain_gpt_2Node4GPU_increase_BS_fp16_activation_checkpointing_gradient_accumulation.sh

# Check the SLURM queue
!squeue

Submitted batch job 15
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                15  slurmpar dli_2nod    admin  R       0:01      2 slurmnode[1-2]


Megatron GPT3 사전 훈련의 성능을 이해하기 위해 생성된 [로그](./megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing_gradient_accumulation.txt) 를 확인하고 강사와 논의해 봅니다.

그래디언트 축적을 활성화하거나 비활성화할 때 GPU당 마이크로 배치 수를 비교해 보겠습니다. 비교를 위해서는 그래디언트 누적이 없는 이전 실행 줄과 비교해 보겠습니다.

In [15]:
print("Without gradient accumulation:")
!grep constant /dli/megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing.txt

print("With 4 gradient accumulation:")
!grep constant /dli/megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing_gradient_accumulation.txt

Without gradient accumulation:
setting number of micro-batches to constant 1
With 4 gradient accumulation:
setting number of micro-batches to constant 4


In [16]:
!grep elapsed /dli/megatron/logs/log_2Nodes4GPUS_increase_GBS_fp16_activation_checkpointing_gradient_accumulation.txt

### 메모리는 어떤가요?

프로파일링은 `run_fp16_activation_checkpointing_gradient_accumulation_gpu0` 실행 시 생성된 텐서보드 링크의  `pytorch_profiler` 탭에서 확인 가능합니다.

우리는 메모리 추적에서 단계당 4개의 그레디언트 누적 단계들을 볼 수 있습니다. 

<img src="images/profiling_FP16_checkpoiting_gradient_acc_memory.png" width="900"/>

좋습니다! 다음으로 넘어가기 전에 이전 실행에서 생성된 불필요한 체크포인트를 삭제하여 디스크 공간을 확보해 보겠습니다.

In [17]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/* 

---
# 4.4 학습 성능

합리적인 시간 내에 대규모 신경망을 훈련시키기 위해서는 인프라 확장이 불가피합니다. 
먼저 트랜스포머 모델의 매개 변수 수를 연산하고 하드웨어 인프라와 실험적으로 관찰된 훈련 처리량에 따라 학습 성능을 추정해 보겠습니다.


## 4.4.1 트랜스포머 모델의 매개 변수 수 연산

트랜스포머 모델의 파라미터 수는 다음과 같이 계산됩니다:

$P = 12 l h^2 (1 + \frac{13}{12h} + \frac{V+s}{12lh})$ where:
- $l$ = Number of Layers
- $h$ = Hidden Size 
- $V$ = Vocabulary Size 
- $s$ = Sequence Length

In [18]:
# Number of parameters of the Transformers model
def calculate_number_parameters(l,h,s,V):
    # Compute the number of parameters of the model
    P=12*l*h*h *(1+ (13/(12*h)) + ((V+s)/(12*l*h)))
    print("The number of parameters for the GPT architecture is: {} \n".format(int(P)))
    return P

예를 들어, 40개의 레이어, 6144의 히든 사이즈, 50257의 vocabulary 크기, 1024의 시퀀스 길이를 가진 트랜스포머 모델의 매개 변수 수를 계산해 보겠습니다. 계산에 따르면 이 모델은 약 180억 개의 매개 변수여야 합니다.

In [19]:
# Set the model architecture parameters
l=40
h=6144
s=1048
V=50257
    
P=calculate_number_parameters(l,h,s,V)

The number of parameters for the GPT architecture is: 18437806080 



## 4.4.2 연습: 모델의 매개 변수 수 연산

이전 노트북에서 실험한 모델의 매개 변수 수를 연산합니다.
이전 노트북의 Megaton-LM GPT 사전 훈련 스크립트에서 모델 아키텍처 인자들을 살펴보십시오. 

막히면 [해설서](solutions/ex4.1.2.ipynb)를 확인할 수 있습니다.

In [21]:
# Set our model architecture parameters
l=40
h=6144
s=1048
V=50257
    
calculate_number_parameters(l,h,s,V)

The number of parameters for the GPT architecture is: 18437806080 



18437806080.0

## 4.4.3 GPU당 이론상 최대 FLOP/초 연산

[Scaling Language Model Training to a Trillion Parameters Using Megatron](https://arxiv.org/pdf/2104.04473.pdf) 논문에서 자세히 설명한 바와 같이 모델에서 대부분의 부동소수점 연산은 행렬 곱(GEMMs)에서 수행됩니다. 이러한 GEMMs 연산만 고려할 경우 반복당 FLOP 수는 다음과 같습니다. 

$F = 96 B s l h^2 (1 + \frac{s}{6h} + \frac{V}{16lh})$ where $B$ is the batch size. 

그리고 반복당 소요 시간`Time_per_iteration_second` 에 대한 추정치가 있는 경우, 초당 및 GPU당 이론 상 최대 FLOP를 계산하고 비교함으로써 GPU 사용량을 추정할 수 있습니다. 

다음 표는 A100 GPU로 구성된 SuperPOD 클러스터에서 Megatron-LM  라이브러리를 사용하여 사전 훈련을 받은 여러 GPT 모델들의 크기별(1.7B~1조)의 훈련 성능을 보여줍니다.
<img src="https://github.com/NVIDIA/Megatron-LM/blob/main/images/cases_april2021.png?raw=true"/>



In [22]:
# Theoretical peak FLOP per second per GPU - with activation checkpointing (2 forwards and 1 backward)
def calculate_Theoretical_peak_FLOP_s_GPU(B,s, l,h,number_GPUs,Time_per_iteration_second):
    # The number of FLOPs per iteration
    F = 96*B*s* l*h*h *(1 + s/ (6*h) + V/(16*l*h))/1e+12
    
    #Theoretical peak FLOP per second per GPU
    PF= (F/Time_per_iteration_second/number_GPUs)
    print("Theoretical peak FLOP/s/GPU: {}\n".format(PF))
    
    # Percentage of theoretical peak FLOP/s on a A100 FP16 (change according the hardware)
    GPU_usage= PF/ 312 *100
    print("Percentage of theoretical peak FLOP/s: {}%".format(GPU_usage))
    
    return PF, GPU_usage

이전 기능에서 이론상 최대 FLOP/s의 백분율은 **FP16/BF16 = 312** 의 **A100** 하드웨어 기능을 기반으로 합니다. 또한 해당 텐서 코어 GPU 성능 사양에 따라 업데이트 될 필요가 있습니다. [Ampere architecture specifications](https://developer.nvidia.com/blog/nvidia-ampere-architecture-in-depth/) 문서에서 더 자세히 알아보세요.



글로벌 배치 크기가 512인 16개의 GPU에서 사전 훈련된 이전 18B 매개 변수 모델을 고려하면 반복당 시간이 32.09초입니다. 
GPU 당 이론상 최대 초당 FLOP와 GPU 활용률을 계산해 보도록 하겠습니다.

In [23]:
global_batch_size=512
number_GPUs=16
Time_per_iteration_second=32.09

# Considering the 18B parameters model
l=40
h=6144
s=1048
V=50257
    
PF,GPU_usage=calculate_Theoretical_peak_FLOP_s_GPU(global_batch_size,s, l,h,number_GPUs,Time_per_iteration_second)

Theoretical peak FLOP/s/GPU: 157.7296879710454

Percentage of theoretical peak FLOP/s: 50.55438717020686%


## 4.4.4 에폭 당 훈련 시간 (Training  Duration / Epoch) 추정

모델, 데이터 세트 및 하드웨어 크기에 따라 에폭 당 훈련 시간을 추정할 수 있습니다. 훈련 시간(초)은 아래의 방정식으로 근사합니다.

Training time (sec) $\approx \frac{8*T*P}{n * PF}$ where: 
- $T$ = Number of tokens in the dataset
- $P$ = Numbers of parameters 
- $n$ = Number of GPUs
- $PF$ = Achieved teraFLOP/s per GPU

자세한 내용은 [[Scaling Language Model Training to a Trillion Parameters Using Megatron](https://arxiv.org/pdf/2104.04473.pdf) 논문에서 확인할 수 있습니다.

다음 2개의 셀을 실행하여 $T$=3000억 토큰의 데이터 세트에서 훈련된 18B 매개 변수 모델에 대한 훈련 시간을 추정해 보겠습니다.

In [24]:
from termcolor import colored

# Estimate the training time
def estimate_days_needed(T , P , N ,PF):  
    compute_sec=8*T*P/(N*PF*10e12)
    # Convert compute seconds to days
    to_days=round(compute_sec/(3600*24))
    print("This language model will need {} days per epoch.".format(colored(str(to_days),'blue', attrs=['bold'])))

In [25]:
# Number of tokens in the dataset
T=300*10e09

estimate_days_needed(T,P,number_GPUs,PF)

This language model will need 203 days per epoch.


이 결과, 203일이 소요되며, 이는 300B 토큰의 데이터 세트가 있는 16개의 GPU (2개 노드)에서 18B 모델을 훈련하는 데 거의 **7개월**이 소요됩니다! 

이 경우 적절한 시간 내에 모델을 훈련하기 위해 노드 수를 확장하는 것이 불가피합니다. 

예를 들어,  $n$=1024 A100 GPU에서 $T$=3000억 개의 토큰 데이터 세트에 대해 훈련된 $P$=1750억 개의 파라미터를 가진 GPT-3 모델을 생각해 보십시오. 1536의 배치 크기를 사용하여 GPU당 $F$=140 teraFLOP/s를 달성합니다. 따라서 이 모델을 훈련하는 데 필요한 시간은 **34일**입니다.

---
<h2 style="color:green;">축하합니다!</h2>

다음으로 넘어가기 전에 대기열에서 실행 중이거나 대기 중인 작업이 없는지 확인해야 합니다.

In [26]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하고 `scancel` 명령을 사용하여 모든 어드민 사용자의 작업을 취소합니다.

In [27]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


다음 노트북에서는 대규모 신경망을 훈련하는 데 사용되는 다른 기술을 실험하고 컴퓨터 비전을 위한 사용법을 시연할 것입니다.